In [ ]:
import matplotlib.pyplot as plt
import transform
import request
import datetime

In [ ]:
start_date_2018 = datetime.datetime(2018, 1, 1).isoformat() + 'Z'
stop_date_2018 = datetime.datetime(2018, 12, 31).isoformat() + 'Z'
start_date_2019 = datetime.datetime(2019, 1, 1).isoformat() + 'Z'
stop_date_2019 = datetime.datetime(2019, 12, 31).isoformat() + 'Z'
start_date_2020 = datetime.datetime(2020, 1, 1).isoformat() + 'Z'
stop_date_2020 = datetime.datetime.utcnow().isoformat() + 'Z'

In [ ]:
df_2018 = transform.get_dataframe(request.get_filtered_events(
    start_date_2018, stop_date_2018, "summary"))
df_2019 = transform.get_dataframe(request.get_filtered_events(
    start_date_2019, stop_date_2019, "summary"))
df_2020 = transform.get_dataframe(request.get_filtered_events(
    start_date_2020, stop_date_2020, "summary"))

In [ ]:
df_2020

Overall distance

In [ ]:
print(df_2020.sum()["distance"])
print(df_2019.sum()["distance"])
print(df_2018.sum()["distance"])

Average distance

In [ ]:
print(df_2020.mean()["distance"])
print(df_2019.mean()["distance"])
print(df_2018.mean()["distance"])

Longest runs

In [ ]:
df_2020.sort_values(by=["distance"], ascending=False)["distance"][:10]

Daily performance

In [ ]:
_, ax = plt.subplots()
df_2020.plot.scatter("date", "distance", ax=ax)

Monthly performance

In [ ]:
df_2020.groupby("month").sum()["distance"]

In [ ]:
_, ax = plt.subplots()
df_2020.groupby(["month"], as_index=False)["distance"].sum().plot(
    x="month", y="distance", kind="scatter", ax=ax, color="b", label="2020")
df_2019.groupby(["month"], as_index=False)["distance"].sum().plot(
    x="month", y="distance", kind="scatter", ax=ax, color="r", label="2019")
df_2018.groupby(["month"], as_index=False)["distance"].sum().plot(
    x="month", y="distance", kind="scatter", ax=ax, color="y", label="2018")
ax.legend()
ax.set_title("Monthly distance")

In [ ]:
_, ax = plt.subplots()
df_2020.groupby(["month"], as_index=False)["distance"].mean().plot(
    x="month", y="distance", kind="scatter", ax=ax, color="b", label="2020")
df_2019.groupby(["month"], as_index=False)["distance"].mean().plot(
    x="month", y="distance", kind="scatter", ax=ax, color="r", label="2019")
df_2018.groupby(["month"], as_index=False)["distance"].mean().plot(
    x="month", y="distance", kind="scatter", ax=ax, color="y", label="2018")
ax.legend()
ax.set_title("Monthly average distance")

Weekly performances

In [ ]:
_, ax = plt.subplots()
df_2020.groupby(["week"], as_index=False)["distance"].sum().plot(x="week", y="distance", kind="scatter", ax=ax)

Longest duration between two runs

In [ ]:
df_2020.sort_values(by=["date"])
t1 = df_2020["date"][0]
max_diff = t1 - t1
max_diff_t = t1
for index, row in df_2020.iterrows():
    t2 = row["date"]
    if t2 - t1 > max_diff:
        max_diff_t = t1
        max_diff = t2 - t1
    t1 = t2
print(max_diff_t)
print(max_diff)

How many weeks without running?

In [ ]:
52 - df_2020["week"].nunique()